# Imports

In [ ]:
# time management
import datetime

# data import, handling, analysis
import pandas
import seaborn
import os

# data visualisation
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# report production
from reportlab.pdfgen import canvas 
from reportlab.pdfbase.ttfonts import TTFont 
from reportlab.pdfbase import pdfmetrics 
from reportlab.lib import colors 
from reportlab.lib.pagesizes import A4


# Load Data

In [ ]:
data = pandas.read_csv('data.csv')
data["Date"] = pandas.to_datetime(data["Date"])
data.set_index("Date", inplace = True)
data.sort_index()

## Data Aggregation

In [ ]:
data["PetrolPrice(PerLitre)"] = data["TotalCost"]/data["PetrolFilled(Litres)"]
data["Kilometrage(L/100km)"] = (data["PetrolFilled(Litres)"]/data["Distance"])*100
data["CostPerKilometre"] = data["TotalCost"]/data["Distance"]

# Idea

Ideas to include:

- Average Monthly Numeric Statistics 
  - Cost
  - Distance Travelled
  - Fuel Efficiency (L/100)
  - Petrol Filled (Litres)
  - Cost per Kilometre
Also Calculate for YTD and compare


- Expected Statistics for the year
  - expected total distance travelled
  - expected total fuel usage (Litres)
  - expected total cost
  - expected environmental impact (estimate)???



- Visualisation of Statistics
  - Cost of Petrol Over Time
  - Proportion of each petrol type used (ULP91 - ULP98)
  - Fuel Usage Over Time
  - Weekly Cost Distribution
  - Cost vs. Distance
  - Fuel Efficiency Over Time
Also Calculate for YTD and compare where appropriate



- output specified report as a PDF
  - DATE PREPARED
  - WATERMARK ([Zec-Wicks 2024](https://zecwicks.com))
  - Concise introduction paragraph
  - Input specifications
    - date range
    - resulting entries
    - etc.
  - Statistics / Data / Visualations within various sections.
  - Conclusion? 


THE DATA IN THE REPORT SHOULD BE SPLIT INTO CURRENT MONTH (SAME DAY FROM PREVIOUS MONTH UNTIL NOW), AND YEAR TO DATE YTD

# Numeric Statistic Calculation

In [ ]:
def numericStatistics(data, startDate = None, endDate = None, function = "sum", monthly = False):
    function = function.lower()

    startDate = data.index.min() if startDate is None else pandas.to_datetime(startDate, dayfirst=True)
    endDate = data.index.max() if endDate is None else pandas.to_datetime(endDate, dayfirst=True)

        
    tmp = data[(startDate <= data.index) & (data.index <= endDate)]

    if monthly:
        tmp = tmp.groupby(tmp.index.to_period("M"))

    if function == "sum":
        return tmp[["Distance", "PetrolFilled(Litres)", "TotalCost"]].sum()
    
    elif function == "average":
        result = tmp[["Distance", "PetrolFilled(Litres)", "TotalCost", "PetrolPrice(PerLitre)", "Kilometrage(L/100km)", "CostPerKilometre"]].mean()

        # Compute the average time between entries (only for non-monthly aggregation)
        if not monthly:
            if len(tmp.index) > 1:  # Ensure there are at least two entries
                averageTimeDifferences = tmp.index.to_series().diff().dt.total_seconds().mean()  # Average Time differences in seconds
                result["AverageTimeBetweenEntries(seconds)"] = pandas.to_timedelta(averageTimeDifferences, unit='s')
            else:
                result["AverageTimeBetweenEntries(seconds)"] = None  # Not enough data points to compute

        return result
    
    elif function == "mode":
        if monthly:
            # Use 'agg' with a lambda function for grouped mode
            return tmp[["PetrolType"]].agg(lambda group: group.mode())
        else:
            # Directly compute mode for non-grouped data
            return tmp[["PetrolType"]].mode()

    else:
        return "Invalid function"


# Data Visualisations

In [ ]:
def dataVisualisations(data):
    figPaths = {}
    
    # Average Daily Distance Travelled
    plt.figure(figsize=(8, 4))
    data.groupby(data.index.date)['Distance'].mean().plot(title='Average Daily Distance Travelled', color='blue')
    plt.xlabel('Date')
    # Rotate x-axis labels
    plt.xticks(rotation=45)
    plt.ylabel('Distance (KM)')
    figPaths['daily_distance'] = "./outputs/daily_distance.png"
    plt.savefig(figPaths['daily_distance'], bbox_inches='tight')
    plt.close()
    

    
    # Money Spent Over Time
    plt.figure(figsize=(8, 4))
    data.groupby(data.index.date)['TotalCost'].sum().plot(title='Money Spent Over Time', color='green')
    plt.xlabel('Date')
    # Rotate x-axis labels
    plt.xticks(rotation=45)
    plt.ylabel('Total Cost ($)')
    figPaths['money_spent'] = "./outputs/money_spent.png"
    plt.savefig(figPaths['money_spent'], bbox_inches='tight')
    plt.close()

    # Fuel Economy Over Time
    plt.figure(figsize=(8, 4))
    seaborn.lineplot(x=data.index, y='Kilometrage(L/100km)', data=data, hue="PetrolType")
    plt.title('Fuel Economy Over Time')
    plt.xlabel('Date')
    # Rotate x-axis labels
    plt.xticks(rotation=45)
    plt.ylabel('Fuel Economy (L/100KM)')
    figPaths['fuel_economy'] = "./outputs/fuel_economy.png"
    plt.savefig(figPaths['fuel_economy'], bbox_inches='tight')
    plt.close()
    
    return figPaths

# Predictive Data Modelling

In [ ]:
def predictiveModelling(data, predictionStartDate):
    fileOutputs = []

    

    return fileOutputs

# Report Generation

In [ ]:
pastMonthEnd = datetime.datetime.now()
pastMonthStart = pastMonthEnd - pandas.to_timedelta("28day")

currentMonthTotals = numericStatistics(data,pastMonthStart,pastMonthEnd,"sum",False)
currentMonthAverages = numericStatistics(data,pastMonthStart,pastMonthEnd,"average",False)

# Data this calendar year
calendarYearStart = datetime.datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0)
calendarYearEnd = datetime.datetime.now().replace(month=12, day=31, hour=23, minute=59, second=59)
calendarYearTotals = numericStatistics(data,calendarYearStart,calendarYearEnd,"sum",False)
calendarYearAverages = numericStatistics(data,calendarYearStart,calendarYearEnd,"average",False)
# predictiveModelling

# Data all time
allTimeSums = numericStatistics(data,function="sum")
allTimeAverages = numericStatistics(data,function="average")

allTimeMonthlySums = numericStatistics(data,function="sum",monthly=True)
allTimeMonthlyAverages = numericStatistics(data,function="average",monthly=True)

In [ ]:
def produceReport(data, reportTitle):
    pdfFilename = './outputs/fuel_consumption_report.pdf'
    c = canvas.Canvas(pdfFilename, pagesize=A4)
    width, height = A4
    y = height - 50  # Initial position
    lineSpacing = 20

    def drawLine(text):
        nonlocal y
        if y < 100:
            c.showPage()
            y = height - 50
        c.drawString(50, y, text)
        y -= lineSpacing
    
    # Current month Report
    c.setFont("Helvetica-Bold", 18)
    drawLine(f"{datetime.datetime.now().strftime('%B')} {datetime.datetime.now().year} Fuel Consumption Report")

    # Current Month Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${currentMonthTotals['TotalCost']:.2f}")
    drawLine(f"Total Distance: {currentMonthTotals['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {currentMonthTotals['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(currentMonthAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(currentMonthAverages['CostPerKilometre'], 2)}")
    drawLine("")

    # Current Year Report
    c.setFont("Helvetica-Bold", 18)
    drawLine(f"{datetime.datetime.now().year} Fuel Consumption Report")

    # Current Year Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${calendarYearTotals['TotalCost']:.2f}")
    drawLine(f"Total Distance: {calendarYearTotals['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {calendarYearTotals['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(calendarYearAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(calendarYearAverages['CostPerKilometre'], 2)}")
    drawLine("")

    # All Time Report
    c.setFont("Helvetica-Bold", 18)
    drawLine("All-Time Fuel Consumption Report")
    
    # All Time Report Content
    c.setFont("Helvetica", 12)
    drawLine(f"Total Cost: ${allTimeSums['TotalCost']:.2f}")
    drawLine(f"Total Distance: {allTimeSums['Distance']:.2f} KM")
    drawLine(f"Total Petrol Consumption: {allTimeSums['PetrolFilled(Litres)']:.2f} L")
    drawLine(f"Fuel Economy: {round(allTimeAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    drawLine(f"Cost Per Kilometre: ${round(allTimeAverages['CostPerKilometre'], 2)}")

    c.showPage()
    y = height - 100  # Initial position

    # Load and Add Charts
    figPaths = dataVisualisations(data)
    
    for figPath in figPaths.values():
        if os.path.exists(figPath):
            if y < 200:
                c.showPage()
                y = height - 50
            c.drawImage(figPath, 50, y - 150, width=400, height=200)
            y -= 230
    
    c.save()
    print(f"Report saved to {pdfFilename}")
    return pdfFilename

# Run visualizations and generate the report
produceReport(data, "Fuel Consumption Report")

In [ ]:
def produceReport(data, reportTitle):

    # Data from last 28 days (4 weeks)
    pastMonthEnd = datetime.datetime.now()
    pastMonthStart = pastMonthEnd - pandas.to_timedelta("28day")
    
    currentMonthTotals = numericStatistics(data,pastMonthStart,pastMonthEnd,"sum",False)
    currentMonthAverages = numericStatistics(data,pastMonthStart,pastMonthEnd,"average",False)


    # Data this calendar year
    calendarYearStart = datetime.datetime.now().replace(month=1, day=1, hour=0, minute=0, second=0)
    calendarYearEnd = datetime.datetime.now().replace(month=12, day=31, hour=23, minute=59, second=59)

    calendarYearTotals = numericStatistics(data,calendarYearStart,calendarYearEnd,"sum",False)

    # predictiveModelling
    

    # Data all time
    allTimeSums = numericStatistics(data,function="sum")
    allTimeAverages = numericStatistics(data,function="average")

    allTimeMonthlySums = numericStatistics(data,function="sum",monthly=True)
    allTimeMonthlyAverages = numericStatistics(data,function="average",monthly=True)

    # dataVisualisationFiles = dataVisualisations()

    # put results into PDF form using reportlab

    # print as a placeholder
    print(f"{datetime.datetime.now().strftime('%B')}, {datetime.datetime.now().year}")
    print("Fuel Consumption Report")

    print(f"\n\n{datetime.datetime.now().strftime('%B')}, {datetime.datetime.now().year} Report")
    print(f"{datetime.datetime.now().strftime('%B')} Totals")
    print(f"\tCurrent Total Cost: ${currentMonthTotals['TotalCost']}")
    print(f"\tCurrent Total Distance Travelled: {round(currentMonthTotals['Distance'],2)} KM")
    print(f"\tCurrent Total Petrol Consumption: {currentMonthTotals['PetrolFilled(Litres)']} L")

    print(f"\n{datetime.datetime.now().strftime('%B')} Average")
    print(f"\tFuel Economy: {round(currentMonthAverages['Kilometrage(L/100km)'],2)}L / 100 KM")
    print(f"\tCost Per Kilometre: ${round(currentMonthAverages['CostPerKilometre'], 2)}")
    print(f"\tTime Between Refills: {currentMonthAverages['AverageTimeBetweenEntries(seconds)'].days} Days")

    print("\n\nAll Time Report")
    print("\tOverall Totals:")
    print(f"\t\tTotal Cost: ${allTimeSums['TotalCost']}")
    print(f"\t\tTotal Distance: {allTimeSums['Distance']} KM")
    print(f"\t\tTotal Petrol Consumption: {allTimeSums['PetrolFilled(Litres)']} L")
    print("\tOverall Averages:")
    print(f"\t\tAverage Cost: ${round(allTimeAverages['TotalCost'],2)}")
    print(f"\t\tAverage Distance: {round(allTimeAverages['Distance'],2)} KM")
    print(f"\t\tAverage Petrol Consumption: {round(allTimeAverages['PetrolFilled(Litres)'],2)} L")
    print(f"\t\tAverage Fuel Economy: {round(allTimeAverages['Kilometrage(L/100km)'],2)}L / 100KM")

    print(f"\n\nYear ({datetime.datetime.now().year}) to Date Report")
    print("\tYear to Date Totals:")
    print(f"\t\tTotal Cost: ${calendarYearTotals['TotalCost']}")
    print(f"\t\tTotal Distance: {calendarYearTotals['Distance']} KM")
    print(f"\t\tTotal Petrol Consumption: {round(calendarYearTotals['PetrolFilled(Litres)'],2)} L")
    
    return

produceReport(data, "e")

PDF Format Idea:
- Current Month Totals
  - Cost
  - Distance
  - Petrol Consumption
- Current Month Stats (Average)
  - Fuel Economy
  - Cost Per Kilometre

- Average Information (AllTimeAverages)
  - Averate Time Between Filling Tank
  - Average Cost
  - Average Distance
  - Average Petrol Consumption

- Average Information
  - Average Fuel Economy
  - Average Cost Per Kilometre

- Year to Date Information
  - Average Cost
  - Average Distance
  - Average Petrol Consumption
  - Average Fuel Economy
  - Average Cost Per Kilometre


- Predictive Analysis
  - Fuel Consumption
  - Costs
  - Distance



- Data Visualisations
  - Monthly Fuel Consumption - Bar Graph
  - Monthly Cost - Bar Graph
  - Yearly Fuel Consumption - Line Graph
  - Yearly Cost - Line Graph
  - Fuel Type Usage Distribution - Pie Chart
  - Petrol Price Over Time (Grouped by Petrol Type) - Line Graph


In [ ]:
produceReport(data, "title")